In [1]:
import pandas as pd

In [2]:

class SHELF :
    def __init__(self) :
        self.products = []
    
    # 선반에 상품을 적재함
    def push(self, product) :
        self.products.append(product)
        
    # 선반에 상품을 추출함 (선반에 해당 상품이 2개 이상 있는 경우 가장 먼저 들어온 상품을 추출함)
    def pop(self, product) :
        for i in range(len(self.products)) :
            if self.products[i].code == product.code :
                del self.products[i]
                return
    # 선반에 물건이 몇개 들어있는 지 알려준다.
    def stuffed(self) :
        return len(self.products)
    
    #데이터 보여주기용 함수
    def show(self) :
        if len(self.products) == 0 :
            print("EMPTY")
        for product in self.products :
            print(product)
        print()
        
        
    
class PRODUCT :
    #  현재 단계에서 잘모르는 데이터에 대해서는 defalut 값으로 처리함
    def __init__(self, code, name = '알수없음', ABC = '.', x = '.', y = '.', z = '.', g = '.') :
        self.code = code
        self.name = name
        self.ABC = ABC
        self.x = x
        self.y = y
        self.z = z
        self.g = g
    def __repr__(self) :
        return f"code = {self.code}, name = {self.name}"
        

class MAP :
    # 워크스테이션 당 맡게 되는 선반의 개수를 일관되게 유지하기 위해서
    # 가로, 세로가 n,m 으로 주어진 경우 공장 크기는 n*8, 10 + m*10 으로 생성됨

    def __init__(self,R=2,C=5,K=987654321) :
        if R < 1 or C < 1: 
            raise ValueError("공장 크기가 적절하지 않습니다.\n")
        R *= 10
        R += 10
        C *= 8
        self.R = R
        self.C = C
        self.K = K
        self.index = 1
        self.map = [[0]*C for _ in range(R)]
        self.products = {}
        
        # 워크스테이션 설정
        for c in range(3,C,8) :
            self.map[3][c] = -1
        for c in range(7,C-4,8) :
            self.map[R-4][c] = -1
            
        # 선반 설정
        break_point = False
        for c in range(1,self.C-2,4) :
            if break_point : break
            for r in range(7,self.R-10,10) :
                if break_point : break
                for dc in range(2) :
                    if break_point : break
                    for dr in range(6) :
                        cr = r + dr
                        cc = c + dc 
                        self.map[cr][cc] = self.index
                        self.index += 1
                        if self.index > K :
                            break_point = True
                            break
        
        # 선반 배열 생성
        self.shelves = [SHELF() for _ in range(self.index)]
            
    # 해당 인덱스를 가진 선반에 물건을 넣는다.
    def push(self, product, index = 0, show = False) :
        # 인덱스가 변수로 주어져 있지 않다면 비어있는 선반을 찾아서 적재한다.
        if index == 0 :
            Full = True
            for i in range(1,self.index) :
                if self.shelves[i].stuffed() < 60 :
                    index = i
                    Full = False
                    break
            if Full :
                print("모든 선반에 물건이 차있기 때문에 적재할 수 없습니다.\n")
                return
            
        # 해당 선반에 물건이 60개 이상이라면 적재하지 않는다.
        if self.shelves[index].stuffed() >= 60 :
            print("f{index} 번 선반이 가득 차 적재할 수 없습니다.")
            return

        # 해당 선반에 물품을 적재한다. 
        self.shelves[index].push(product)
        # 적재된 상품이 몇번째 선반에 있는지 저장한다.
        if product.code in self.products.keys() :
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1
        else :
            self.products[product.code] = [0 for _ in range(self.index)]
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1
        
        if show :
            print(f"{product.name} 이 {index} 번째 선반에 적재되었습니다.\n")
            
    # 해당 인덱스를 가진 선반에서 물건을 뺀다.
    def pop(self, product, index = 0, show = False) :
        # 상품이 공장에 존재하지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return
        
        # 상품이 공장에 존재하나 인덱스가 주어지지 않은 경우 자동으로 인덱스를 구함
        if index == 0 :
            for i,cnt in enumerate(self.products[product.code]) :
                if cnt > 0 :
                    index = i
                    break
        
        # 상품이 공장에 존재하고 인덱스가 주어진 경우인데도 해당 선반에 상품이 존재하지 않은 경우
        if self.products[product.code][index] == 0 :
            print(f"{product.name} 이 해당 선반에 존재하지 않습니다.")
            return
        
        # 상품을 추출함
        if show :
            print(f"{product.name} 이 {index} 번째 선반에 추출되었습니다.\n")
        self.products[product.code][index] -= 1
        self.products[product.code][0] -= 1
        if self.products[product.code][0] == 0 :
            del self.products[product.code]
        self.shelves[index].pop(product)
        
    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 불러와 맵에 적재해 줌
    def push_orders(self, name, head = 0, show = False) :
        order_data = pd.read_csv(name)
        if head > 0 :
            order_data = order_data[:head]
        columns = ['상품코드', '상품', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)','중량(g)']
        for i in range(len(order_data)) :
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]], order_data.loc[i][columns[2]], 
                            order_data.loc[i][columns[3]], order_data.loc[i][columns[4]], order_data.loc[i][columns[5]], 
                            order_data.loc[i][columns[6]])
            self.push(product, show = show)
    
    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 풀러와 맵에서 추출해 줌
    def pop_orders(self, name, head = 0, show = False) :
        order_data = pd.read_csv(name)
        if head > 0 :
            order_data = order_data[:head]
        columns = ['상품코드', '상품', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)','중량(g)']
        for i in range(len(order_data)) :
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]], order_data.loc[i][columns[2]], 
                            order_data.loc[i][columns[3]], order_data.loc[i][columns[4]], order_data.loc[i][columns[5]], 
                            order_data.loc[i][columns[6]])
            self.pop(product, show = show)
            
    # 상품이 몇번 선반에 들어있는지 알려준다.
    def show_product(self, product) :
        # 상품이 공장에 적재되어있지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return
        
        print(product, f", 총 {self.products[product.code][0]} 개")
        for index,cnt in enumerate(self.products[product.code][1:]) :
            if cnt == 0 :
                continue
            print(f"{index+1}번 선반 - {cnt} 개")
        print()
    
             
    # 해당 인덱스를 가진 선반이 어떤 물건을 가지고 있는지 보여준다.
    def show_shelf(self, index) :
        print(f"{index} 번째 선반에 들어있는 상품 : ")
        self.shelves[index].show()

    
    # MAP의 형태를 보여준다. W = workstation, @ = 선반
    # index = True 일 때는 선반의 인덱스를 보여준다. 1,2,3... = 선반의 index, -1 = workstation
    def show(self, index = False) :
        print(f"R : {self.R} C : {self.C}\n")
        
        if index :
            for r in range(self.R) :
                for c in range(self.C) :
                    print(str(self.map[r][c]).ljust(len(str(self.index))), end = ' ')
                print()
            return
        for r in range(self.R) :
            for c in range(self.C) :
                if self.map[r][c] == -1 :
                    print('W', end = ' ')
                elif self.map[r][c] > 0 :
                    print("@", end = ' ')
                else : print('-', end = ' ')
            print("\n")


# 사용예시

## 공장 생성

In [3]:
# 맵 생성
# 인풋 : 공장의 행 개수, 열 개수, 선반의 개수(default = 넣을 수 있는 최대 선반 개수)
# 공장 크기는 10+열 개수*10 X 행 개수 * 8 로 생성된다.

M = MAP(1,1)


In [4]:

# 공장의 형태 확인
# @ = 선반, W = 워크스테이션
M.show()

R : 20 C : 8

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - W - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 



In [5]:

# 1이상의 자연수 = 선반 인덱스, -1 = 워크스테이션
M.show(index = True)

R : 20 C : 8

0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  -1 0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  1  7  0  0  13 19 0  
0  2  8  0  0  14 20 0  
0  3  9  0  0  15 21 0  
0  4  10 0  0  16 22 0  
0  5  11 0  0  17 23 0  
0  6  12 0  0  18 24 0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  


## 상품 적재

- 상품 간 구별 기준은 상품 코드 하나뿐임.
- 이는 2개의 상품이 상품코드만 다르고 다른 특성이 모두 일치하여도 다른 상품으로 구분한다는 뜻이다.
- 반대로 상품코드만 정확히 알고 있다면 상품을 적재, 추출할 수 있다.

In [6]:
# 상품을 적재한다.
# 인풋 : 상품 클래스(필요 변수 : 상품 코드), 적재할 선반 인덱스(default = 비어있는 선반의 인덱스를 자동으로 찾는다.)
# show = True 라면 적재되었다는 확인 메세지를 출력해줍니다.
M.push(PRODUCT("001"))
M.push(PRODUCT("001"), index = 10, show = True)
M.push(PRODUCT("001", "치약"), show = True)


알수없음 이 10 번째 선반에 적재되었습니다.

치약 이 1 번째 선반에 적재되었습니다.



In [7]:
# 상품 데이터(csv)를 받아서 적재한다
# 인풋 : 상품 데이터(csv) 파일의 경로와 이름
M.push_orders('order_data.csv', head = 10, show = True)


히말라야 핑크솔트 이 1 번째 선반에 적재되었습니다.

샤프란 향신료 20g 이 1 번째 선반에 적재되었습니다.

Elegance Plus Shaving Gel 500ml 이 1 번째 선반에 적재되었습니다.

원두커피: 3. 라바짜 그란에스프레소 이 1 번째 선반에 적재되었습니다.

악역의 엔딩은 죽음뿐 마스킹 테이프 vol.2 이 1 번째 선반에 적재되었습니다.

PET BED-Grey 385 이 1 번째 선반에 적재되었습니다.

(1매) 프리스틴 KF94 마스크(새부리형/화이트) 이 1 번째 선반에 적재되었습니다.

벨 숄더_VANILLA CREAM 이 1 번째 선반에 적재되었습니다.

모르간 버킷 BLACK XXX 이 1 번째 선반에 적재되었습니다.

모르간 버킷 BLACK XXX 이 1 번째 선반에 적재되었습니다.



## 공장에 적재된 상품 데이터 확인

In [8]:

# 선반에 적재된 상품 확인 - 인풋 : 선반의 인덱스
# 해당 선반에 적재되어있는 상품 정보를 알려줌
M.show_shelf(1)
M.show_shelf(10)


1 번째 선반에 들어있는 상품 : 
code = 001, name = 알수없음
code = 001, name = 치약
code = 802980111116, name = 히말라야 핑크솔트
code = 6261108117326, name = 샤프란 향신료 20g
code = 5285001951833, name = Elegance Plus Shaving Gel 500ml
code = 8000070021341, name = 원두커피: 3. 라바짜 그란에스프레소
code = VD-MT-002, name = 악역의 엔딩은 죽음뿐 마스킹 테이프 vol.2
code = 0081010, name = PET BED-Grey 385
code = 8809764510020, name = (1매) 프리스틴 KF94 마스크(새부리형/화이트)
code = O2SBSD39130XXX, name = 벨 숄더_VANILLA CREAM
code = O0SBBC47009XXX, name = 모르간 버킷 BLACK XXX
code = O0SBBC47009XXX, name = 모르간 버킷 BLACK XXX

10 번째 선반에 들어있는 상품 : 
code = 001, name = 알수없음



In [9]:
# 상품이 적재된 선반 확인 - 인풋 : 상품 클래스(필요 변수 : 상품 코드)
M.show_product(PRODUCT("001", "치약"))

code = 001, name = 치약 , 총 3 개
0번 선반 - 2 개
9번 선반 - 1 개



## 상품 추출

In [10]:

# 상품 추출 - 인풋 : 상품 클래스(필요 변수 : 상품 코드),
#   추출할 선반의 인덱스(default : 상품이 적재된 선반의 index 를 자동으로 찾는다.)
M.pop(PRODUCT("001", "치약"), show = True)
M.pop(PRODUCT('001'), show = True)
M.pop(PRODUCT('001'), index = 10, show = True)


치약 이 0 번째 선반에 추출되었습니다.

알수없음 이 0 번째 선반에 추출되었습니다.

알수없음 이 10 번째 선반에 추출되었습니다.



In [11]:
# 상품 데이터(csv) 파일을 받아서 추출한다.
# 인풋 : 상품 데이터(csv) 파일의 경로와 이름
M.pop_orders('order_data.csv', head = 10, show = True)

히말라야 핑크솔트 이 0 번째 선반에 추출되었습니다.

샤프란 향신료 20g 이 0 번째 선반에 추출되었습니다.

Elegance Plus Shaving Gel 500ml 이 0 번째 선반에 추출되었습니다.

원두커피: 3. 라바짜 그란에스프레소 이 0 번째 선반에 추출되었습니다.

악역의 엔딩은 죽음뿐 마스킹 테이프 vol.2 이 0 번째 선반에 추출되었습니다.

PET BED-Grey 385 이 0 번째 선반에 추출되었습니다.

(1매) 프리스틴 KF94 마스크(새부리형/화이트) 이 0 번째 선반에 추출되었습니다.

벨 숄더_VANILLA CREAM 이 0 번째 선반에 추출되었습니다.

모르간 버킷 BLACK XXX 이 0 번째 선반에 추출되었습니다.

모르간 버킷 BLACK XXX 이 공장에 존재하지 않습니다.



In [12]:
# 추출되었는 확인
M.show_shelf(1)
M.show_shelf(10)
M.show_product(PRODUCT("001", "치약"))

1 번째 선반에 들어있는 상품 : 
code = 001, name = 알수없음
code = 001, name = 치약
code = 802980111116, name = 히말라야 핑크솔트
code = 6261108117326, name = 샤프란 향신료 20g
code = 5285001951833, name = Elegance Plus Shaving Gel 500ml
code = 8000070021341, name = 원두커피: 3. 라바짜 그란에스프레소
code = VD-MT-002, name = 악역의 엔딩은 죽음뿐 마스킹 테이프 vol.2
code = 0081010, name = PET BED-Grey 385
code = 8809764510020, name = (1매) 프리스틴 KF94 마스크(새부리형/화이트)
code = O2SBSD39130XXX, name = 벨 숄더_VANILLA CREAM
code = O0SBBC47009XXX, name = 모르간 버킷 BLACK XXX
code = O0SBBC47009XXX, name = 모르간 버킷 BLACK XXX

10 번째 선반에 들어있는 상품 : 
EMPTY

code = 001, name = 치약 , 총 -2 개
0번 선반 - 2 개

